## Basic Binary Operators

* The two operands are only computed when called.
* The "\_\_call\_\_" function is implemented to make the operator compatiable with _lambda_ functions.
* Log is for debugging.

In [1]:
import sys, numpy as np

VERBOSE = True

class Log(object):
    def __init__(self, name, verbose=VERBOSE):
        self._name = name
        self._verbose = verbose
    
    def __enter__(self):
        if self._verbose:
            print '%s(' % self._name,
        
    def __exit__(self, type_, value, traceback_):
        if self._verbose:
            print ')',
        return False

class BinaryOp(object):
    def __init__(self, a, b, name='binary_op'):
        self.name = name
        self._a = a
        self._b = b
        
    def compute(self, a, b):
        raise NotImplementedError
        
    def __call__(self, verbose=False):
        with Log(self.name):
            a = self._a()
            b = self._b()
        return self.compute(a, b)

class Add(BinaryOp):
    def __init__(self, a, b, name='add'):
        BinaryOp.__init__(self, a, b, name)
        
    def compute(self, a, b):
        return a + b
    
class Mul(BinaryOp):
    def __init__(self, a, b, name='mul'):
        BinaryOp.__init__(self, a, b, name)
        
    def compute(self, a, b):
        return a * b

## Class for Number

* It can be regarded as a placeholder which is assigned later.

In [2]:
class Num(object):
    def __init__(self, name, v=None):
        self.name = name
        self._v = v
        
    def assign(self, v):
        self._v = v
        
    def read(self):
        if self._v is None:
            raise ValueError('Not initialized number: %s' % self.name)
        return self._v
    
    def __call__(self):
        with Log(self.name):
            v = self.read()
        return v

a = Num('a', 1.)
b = Num('b')
s = Add(a, b)
res = Mul(b, s)

print s
print res

try:
    print res()
except ValueError:
    print sys.exc_info()

b.assign(2.)
print res()

mul( b( ) ) (<type 'exceptions.ValueError'>, ValueError('Not initialized number: b',), <traceback object at 0x0000000005BC9348>)
mul( b( ) add( a( ) b( ) ) ) 6.0


## Class for Array

* The dimension can be assigned later.
* The memory is reallocated when needed.

In [3]:
class Ones(object):
    def __init__(self, name, dim=None):
        self.name = name
        self._dim = Num('dim', dim)
        self._cached = False
        self._cached_arr = None
        self._arr = lambda : np.ones(self._dim(), dtype=np.float32)
    
    def set_dim(self, dim):
        self._dim.assign(dim)
        self._cached = False
        self._cached_arr = None
        
    def read(self):
        try:
            if self._cached:
                return self._cached_arr
            else:
                return self._arr()
        except ValueError:
            print sys.exc_info()
            raise ValueError('Not initialized dim: %s' % self.name)
    
    def __call__(self):
        with Log(self.name):
            self._cached_arr = self.read()
            self._cached = True
        return self._cached_arr
    
dim = 4
c = Ones('c', dim)
d = Ones('d')

s2 = Add(c, d)
res2 = Mul(s2, s2)

print s2
print res2

try:
    print res2()
except ValueError:
    print sys.exc_info()

d.set_dim(dim)
print res2()


dim = 6
c.set_dim(dim)
d.set_dim(dim)
print res2()

mul( add( c( dim( ) ) d( dim( ) (<type 'exceptions.ValueError'>, ValueError('Not initialized number: dim',), <traceback object at 0x0000000005BD67C8>)
) ) ) (<type 'exceptions.ValueError'>, ValueError('Not initialized dim: d',), <traceback object at 0x0000000005BD6708>)
mul( add( c( ) d( dim( ) ) ) add( c( ) d( ) ) ) [ 4.  4.  4.  4.]
mul( add( c( dim( ) ) d( dim( ) ) ) add( c( ) d( ) ) ) [ 4.  4.  4.  4.  4.  4.]


## Class for Conditional Statements

In [4]:
class NoOp(object):
    def __call__(self):
        with Log('no_op'):
            pass

NO_OP = NoOp()
        
        
class Less(BinaryOp):
    def __init__(self, a, b, name='less'):
        BinaryOp.__init__(self, a, b, name)
        
    def compute(self, a, b):
        return a < b
    
class Cond(object):
    def __init__(self, cond, true_op, false_op, name='cond'):
        self.name = name
        self._cond = cond
        self._true_op = true_op
        self._false_op = false_op
        
    def compute(self, cond):
        return self._true_op() if cond else self._false_op()
    
    def __call__(self):
        with Log(self.name):
            cond = self._cond()
            return self.compute(cond)

class WhileLoop(object):
    def __init__(self, cond, body_op, update_op, name='while'):
        self.name = name
        self._cond = cond
        self._body_op = body_op
        self._update_op = update_op
        
    def compute(self, cond):
        if cond:
            self._body_op()
            self._update_op()
            return self()
        else:
            return NO_OP()
    
    def __call__(self):
        with Log(self.name):
            cond = self._cond()
            return self.compute(cond)
        
i = Num(name='i')
one = Num(name='one', v=1.)
n = Num(name='n')
sum_sqr = Num(name='sum_sqr')

less_ = Less(i, n)

def body():
    global i, sum_sqr
    i_v = i.read()
    sqr = Mul(i, i, name='sqr_%d' % i_v)
    sum_ = Add(sum_sqr, sqr, name='sum_%d' % i_v)
    sum_sqr.assign(sum_())

def update():
    global i, one
    i_new = Add(i, one, name='add_%d' % i.read())
    i.assign(i_new())
    
loop_op = WhileLoop(less_, body, update)

print loop_op
try:
    loop_op()
    print i(), sum_sqr()
except ValueError:
    print sys.exc_info()
    
i.assign(0.)
n.assign(10.)
sum_sqr.assign(0.)
loop_op()
print i.read(), sum_sqr.read()

while( less( i( ) ) ) (<type 'exceptions.ValueError'>, ValueError('Not initialized number: i',), <traceback object at 0x0000000005C01888>)
while( less( i( ) n( ) ) sum_0( sum_sqr( ) sqr_0( i( ) i( ) ) ) add_0( i( ) one( ) ) while( less( i( ) n( ) ) sum_1( sum_sqr( ) sqr_1( i( ) i( ) ) ) add_1( i( ) one( ) ) while( less( i( ) n( ) ) sum_2( sum_sqr( ) sqr_2( i( ) i( ) ) ) add_2( i( ) one( ) ) while( less( i( ) n( ) ) sum_3( sum_sqr( ) sqr_3( i( ) i( ) ) ) add_3( i( ) one( ) ) while( less( i( ) n( ) ) sum_4( sum_sqr( ) sqr_4( i( ) i( ) ) ) add_4( i( ) one( ) ) while( less( i( ) n( ) ) sum_5( sum_sqr( ) sqr_5( i( ) i( ) ) ) add_5( i( ) one( ) ) while( less( i( ) n( ) ) sum_6( sum_sqr( ) sqr_6( i( ) i( ) ) ) add_6( i( ) one( ) ) while( less( i( ) n( ) ) sum_7( sum_sqr( ) sqr_7( i( ) i( ) ) ) add_7( i( ) one( ) ) while( less( i( ) n( ) ) sum_8( sum_sqr( ) sqr_8( i( ) i( ) ) ) add_8( i( ) one( ) ) while( less( i( ) n( ) ) sum_9( sum_sqr( ) sqr_9( i( ) i( ) ) ) add_9( i( ) one( ) ) while( less

* while(
  + less( i( ) n( ) )
  + sum_0( sum_sqr( ) sqr_0( i( ) i( ) ) )
  + add_0( i( ) one( ) )
  + while(
    + less( i( ) n( ) )
    + sum_1( sum_sqr( ) sqr_1( i( ) i( ) ) )
    + add_1( i( ) one( ) )
    + while(
      + less( i( ) n( ) )
      + sum_2( sum_sqr( ) sqr_2( i( ) i( ) ) )
      + add_2( i( ) one( ) )
      + while(
        + less( i( ) n( ) )
        + sum_3( sum_sqr( ) sqr_3( i( ) i( ) ) )
        + add_3( i( ) one( ) )
        + while(
          + less( i( ) n( ) )
          + sum_4( sum_sqr( ) sqr_4( i( ) i( ) ) )
          + add_4( i( ) one( ) )
          + while(
            + less( i( ) n( ) )
            + sum_5( sum_sqr( ) sqr_5( i( ) i( ) ) )
            + add_5( i( ) one( ) )
            + while(
              + less( i( ) n( ) )
              + sum_6( sum_sqr( ) sqr_6( i( ) i( ) ) )
              + add_6( i( ) one( ) )
              + while(
                + less( i( ) n( ) )
                + sum_7( sum_sqr( ) sqr_7( i( ) i( ) ) )
                + add_7( i( ) one( ) )
                + while(
                  + less( i( ) n( ) )
                  + sum_8( sum_sqr( ) sqr_8( i( ) i( ) ) )
                  + add_8( i( ) one( ) )
                  + while(
                    + less( i( ) n( ) )
                    + sum_9( sum_sqr( ) sqr_9( i( ) i( ) ) )
                    + add_9( i( ) one( ) )
                    + while(
                      + less( i( ) n( ) )
                      + no_op( )
                    + )
                  + )
                + )
              + )
            + )
          + )
        + )
      + )
    + )
  + )
+ )
10.0 285.0